# **Project Tugas Akhir**

- Nama: Rizky Amalia
- NIM: 19142037P
- Prodi: Teknik Informatika

# Import Library

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import imutils
from imutils.video import VideoStream

# Proses Deteksi Wajah

In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet): # faceNet=face detector; maskNet=masker detection
    # Resize dimensi ke 224 x 224
    (h,w) = frame.shape[:2]

    # Pada tahap ini akan dihitung jumlah rata-rata intensitas pixel pada seluruh gambar untuk setiap warna merah, 
    # hijau dan juga biru. Proses ini bertujuan untuk menyiapkan gambar untuk masuk ke deep neural network
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224),(104.0,117.0,123.0))
    
    # Face Detection
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    # Menampung wajah, lokasi dan hasil prediksi dalam list
    wajah = []
    lokasi = []
    preds = []
    
    # Proses Deteksi Wajah
    for i in range(0, detections.shape[2]):
        confidence = detections[0,0,i,2]

        if confidence > 0.5:
            # Penandaan koordinat X dan Y
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (X_start,Y_start,X_end,Y_end) = box.astype('int') # bounding box

            # Pembuatan bounding box untuk menandai wajah, memastikan bounding box berada di dalam frame
            (X_start,Y_start) = (max(0,X_start), max(0,Y_start))
            (X_end,Y_end)     = (min(w-1,X_end), min(h-1,Y_end))

            # Extract ROI lalu konversi BGR ke RGB, resize gambar ke 224 x 224
            face = frame[Y_start:Y_end, X_start:X_end]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face,(224,224))
            face = img_to_array(face)
            face = preprocess_input(face)

            wajah.append(face)
            lokasi.append((X_start,Y_start,X_end,Y_end))

        # Buat prediksi hanya jika -paling tidak- ada 1 wajah yang terdeteksi
        if len(wajah) > 0:
            wajah = np.array(wajah, dtype='float32')
            preds = maskNet.predict(wajah, batch_size = 12)
        
        return(lokasi, preds)

# Load Detektor Wajah dan Masker

In [3]:
prototxt = os.path.sep.join([r'D:\OneDrive\Kuliah\KULIAH\MATA KULIAH\Semester 5\Project\4 Face Detector',
                             'deploy.prototxt'])

weights  = os.path.sep.join([r'D:\OneDrive\Kuliah\KULIAH\MATA KULIAH\Semester 5\Project\4 Face Detector',
                             'res10_300x300_ssd_iter_140000.caffemodel'])

faceNet = cv2.dnn.readNet(prototxt, weights)
maskNet = load_model(r'D:\OneDrive\Kuliah\KULIAH\MATA KULIAH\Semester 5\Project\Mod_Save2\model_adam\model_adam.model')

# Test Deteksi Masker dengan Video Real Time

In [4]:
vs = VideoStream(src=0).start()

while True:
    # Membuka frame video kemudian resize framenya menjadi ukuran lebar sebesar 600 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width = 650)
    
    # Mendeteksi wajah yang ada di gambar kemudian prediksi apakah sosok dalam frame menggunakan masker atau tidak
    (lokasi, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    for (box, pred) in zip(lokasi, preds):
        (X_start, Y_start, X_end, Y_end) = box
        (Masker, Tidak_Bermasker) = pred
        
        # Menentukan label beserta warna bounding box sesuai labelnya                     
        label = 'Masker' if Masker > Tidak_Bermasker else 'Tidak Bermasker'
        color = (0,255,0) if label == 'Masker' else (0,0,255)
        
        # Menambahkan Persentase pada label                     
        label = "{}: {:.2f}%".format(label, max(Masker, Tidak_Bermasker)*100)
        
        # Menampilkan label dan bounding box
        cv2.putText(frame,label,(X_start, Y_start-10), cv2.FONT_HERSHEY_PLAIN, 0.80, color, 2)
        cv2.rectangle(frame, (X_start, Y_start), (X_end, Y_end), color, 2)
        
    # Menampilkan Frame Output
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('x'):
        break
        
cv2.destroyAllWindows()
vs.stop()